# Reading training dataset from tfrecord file

In [22]:
from getpass import getpass
from mysql.connector import connect, Error

def query(text, inputs=None):
    try: 
        with connect(
            host='localhost',
            user='apalermo01',
            password='pass_2112',
            database='music',
        ) as connection:
            with connection.cursor() as cursor:
                cursor.execute(text, inputs)
                res = []
                for db in cursor:
                    res.append(db)
                return res
    except Error as e:
        print("exception")
        print(e)

In [23]:
show_db_query = "SHOW DATABASES"
test_query = "SHOW TABLES"
test1_query = "SELECT * FROM train LIMIT 10;"
test2 = "DESCRIBE test;"
res = query(test2)
res

[('id', b'int', 'NO', 'PRI', None, 'auto_increment'),
 ('mfcc', b'text', 'YES', '', None, ''),
 ('filename', b'varchar(100)', 'YES', '', None, ''),
 ('folder_id', b'int', 'YES', '', None, ''),
 ('target', b'int', 'YES', '', None, ''),
 ('fft', b'text', 'YES', '', None, '')]

In [6]:
text = """
SELECT id, filename, folder_id, target FROM test LIMIT 10
"""
query(text)

[(20485, 'guitar_acoustic_010-074-100.wav', 1, 219),
 (20486, 'organ_electronic_113-048-075.wav', 1, 958),
 (20487, 'bass_synthetic_098-106-100.wav', 1, 803),
 (20488, 'bass_synthetic_033-100-100.wav', 1, 417),
 (20489, 'keyboard_synthetic_000-095-127.wav', 1, 577),
 (20490, 'bass_synthetic_068-056-075.wav', 1, 656),
 (20491, 'organ_electronic_057-061-100.wav', 1, 701),
 (20492, 'reed_acoustic_011-076-100.wav', 1, 104),
 (20493, 'bass_electronic_025-048-075.wav', 1, 872),
 (20494, 'guitar_acoustic_015-038-075.wav', 1, 316)]

In [8]:
text = """
SELECT id, filename, folder_id, target
FROM test
WHERE
    test.filename Like "guitar%"
LIMIT 10
"""
query(text)

[(20485, 'guitar_acoustic_010-074-100.wav', 1, 219),
 (20494, 'guitar_acoustic_015-038-075.wav', 1, 316),
 (20496, 'guitar_electronic_028-091-127.wav', 1, 510),
 (20499, 'guitar_acoustic_014-037-075.wav', 1, 263),
 (20504, 'guitar_acoustic_021-069-025.wav', 1, 457),
 (20514, 'guitar_electronic_022-054-100.wav', 1, 378),
 (20518, 'guitar_electronic_028-101-100.wav', 1, 510),
 (20527, 'guitar_acoustic_014-104-025.wav', 1, 263),
 (20530, 'guitar_acoustic_021-022-075.wav', 1, 457),
 (20545, 'guitar_acoustic_014-102-100.wav', 1, 263)]

# Develop a pipeline to do batch learning

Get a numpy array with unique ids

In [9]:
text = """
SELECT id FROM test
"""

ids = query(text)

In [16]:
import numpy as np

In [17]:
ids_new = np.array([i[0] for i in ids])

In [20]:
ids_new

array([20485, 20486, 20487, ..., 24578, 24579, 24580])

# Get a sample mfcc / input

In [25]:
id_select = [str(ids_new[0])]

text = """
SELECT mfcc
FROM test
WHERE id = (%s)
"""

mfcc = query(text, id_select)


In [70]:
PATH = "C:/Users/alexc/Downloads/nsynth-train.jsonwav.tar.gz"
import numpy as np
import librosa
import tarfile
import soundfile as sf
import io

def getfiles(PATH, num_files = 1000):
    with tarfile.open(PATH, 'r:gz') as tar:
        index = 0
        #while index < max_index:
        while True:
            fname = tar.next()
            if fname is None: 
                break
            if fname.name.endswith(".wav"):
                wav_file = tar.extractfile(fname).read()
                data, sr = sf.read(io.BytesIO(wav_file))
                yield data, sr, fname.name
data_gen = getfiles(PATH)

Write a function to get several files

In [72]:
test = next(data_gen)

In [77]:
test[2].split('/')[2].split('_')[0]

'bass'

In [78]:
targets_list = [
    "bass", "brass", "flute", "guitar", "keyboard",
    "mallet", "organ", "reed", "string", "synth_lead", "vocal"
]

In [81]:
def getdata(data_gen=data_gen, num_files=10):
    data = []
    for i in range(num_files):
        record = next(data_gen)
        target = record[2].split('/')[2].split('_')[0]
        data.append((record[0], record[1], target))
    return data

In [82]:
results = getdata()

In [83]:
results[0]

(array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'keyboard')

Compute the mfccs

In [87]:
results

[(array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'keyboard'),
 (array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'string'),
 (array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'keyboard'),
 (array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'brass'),
 (array([ 0.        , -0.00094604, -0.00268555, ...,  0.        ,
          0.        ,  0.        ]),
  16000,
  'reed'),
 (array([ 0.00112915, -0.0015564 ,  0.00216675, ...,  0.        ,
          0.        ,  0.        ]),
  16000,
  'keyboard'),
 (array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'string'),
 (array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'keyboard'),
 (array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'guitar'),
 (array([0., 0., 0., ..., 0., 0., 0.]), 16000, 'organ')]

In [93]:
def getmfccs(data_tuple):
    data = data_tuple[0]
    sr = data_tuple[1]
    mfcc = librosa.feature.mfcc(y=data, sr=sr)
    mfccStr = str(list(mfcc))
    return mfcc

Use one-hot encoding for the target values

In [131]:
from sklearn.preprocessing import OneHotEncoder

In [132]:
targets_list

['bass',
 'brass',
 'flute',
 'guitar',
 'keyboard',
 'mallet',
 'organ',
 'reed',
 'string',
 'synth_lead',
 'vocal']

In [133]:
enc = OneHotEncoder()

# Implement a basic network

Get the input shape

In [67]:
from tensorflow import keras

# build the network architecture
model = keras.Sequential([
    # input layer                    # flatten this 2D array, intervals x values of MFCCs for that interval
    keras.layers.Flatten(input_shape=(20, 126)),
    
    # 1st hidden layer  
    keras.layers.Dense(512, activation="relu"), 
    
    # 2nd hidden layer
    keras.layers.Dense(256, activation="relu"), 
    
    # 3rd hidden layer
    keras.layers.Dense(64,  activation="relu"),
    
    # output layers- 1 neuron for each genre
    keras.layers.Dense(10, activation="softmax")
])

# compile network
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2520)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1290752   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 1,439,178
Trainable params: 1,439,178
Non-trainable params: 0
_________________________________________________________________


Train with one batch of data

In [100]:
dataset = getdata()

In [109]:
X = np.array([getmfccs(i) for i in dataset])
t = np.array([targets_list.index(i[2]) for i in dataset])

In [110]:
t

array([4, 4, 1, 5, 0, 5, 0, 8, 8, 7])

In [111]:
type(X)

numpy.ndarray

In [113]:
model.fit(X, t, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 1ms/step - loss: 208.8023 - accuracy: 0.3000
Epoch 2/10
1/1 [==============================] - 0s 1ms/step - loss: 187.9819 - accuracy: 0.3000
Epoch 3/10
1/1 [==============================] - 0s 999us/step - loss: 125.4774 - accuracy: 0.4000
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: 31.1182 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 26.2809 - accuracy: 0.6000
Epoch 6/10
1/1 [==============================] - 0s 2ms/step - loss: 24.8144 - accuracy: 0.7000
Epoch 7/10
1/1 [==============================] - 0s 495us/step - loss: 8.5118 - accuracy: 0.8000
Epoch 8/10
1/1 [==============================] - 0s 11ms/step - loss: 1.8464 - accuracy: 0.8000
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.0629 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 10ms/step - loss: 11.2563 - accuracy: 0.8000


In [127]:
targets_list = [
    "bass", "brass", "flute", "guitar", "keyboard",
    "mallet", "organ", "reed", "string", "synth_lead", "vocal"
]

def getfiles(PATH, num_files = 100):
    with tarfile.open(PATH, 'r:gz') as tar:
        index = 0
        while index < num_files:
            fname = tar.next()
            if fname is None: 
                break
            if fname.name.endswith(".wav"):
                wav_file = tar.extractfile(fname).read()
                data, sr = sf.read(io.BytesIO(wav_file))
                target = fname.name.split('/')[2].split('_')[0]
                yield data, sr, target, targets_list.index(target) 
                index += 1

In [128]:
rec = getfiles(PATH)

In [129]:
res = next(rec)

In [130]:
res

(array([-0.00012207, -0.00445557, -0.00973511, ...,  0.        ,
         0.        ,  0.        ]),
 16000,
 'mallet',
 5)